In [2]:
import pandas as pd
import glob

# location of csv files
csv_directory = 'C:/Users/abhi2/Downloads/SampleData'

# csv files list
csv_files = glob.glob(csv_directory + '/*.csv')

# Initialize an empty list to store the dataframes
dfs = []

# in each CSV file
for file in csv_files:
    # filename must contain 'closePosition'
    if 'closePosition' in file:
        # extracting the required columns
        data = pd.read_csv(file, usecols=['Key', 'ExitTime', 'Symbol', 'EntryPrice', 'Quantity', 'Pnl'])
        
        # Extracting date from 'ExitTime' column and create a new 'Date' column
        data['Date'] = pd.to_datetime(data['ExitTime']).dt.date
        
        # Append the dataframe to the list
        dfs.append(data)

# into a single dataframe
combined_data = pd.concat(dfs, ignore_index=True)

# final dataframe into CSV file
combined_data.to_csv('combined_closePosition.csv', index=False)


In [16]:
import pandas as pd

# csv file into a DataFrame
combined_data = pd.read_csv('combined_closePosition.csv')

# total trades
total_trades = len(combined_data)

# unique days
unique_days = len(combined_data['Date'].unique())

# average trades
average_trades = total_trades / unique_days

# total Pnl
total_pnl = combined_data['Pnl'].sum()

# profit trades
profit_trades = len(combined_data[combined_data['Pnl'] > 0])

# loss trades
loss_trades = len(combined_data[combined_data['Pnl'] <= 0])

# calculated values dictionary
stats = {
    'Total trades': total_trades,
    'Unique days': unique_days,
    'Average trades': average_trades,
    'Total Pnl': total_pnl,
    'Profit Trades': profit_trades,
    'Loss Trades': loss_trades
}

# Savingin a text file
with open('combined_stats.txt', 'w') as file:
    for key, value in stats.items():
        file.write(f"{key}: {value}\n")


In [25]:
import pandas as pd

# Reading the combined_closePosition.csv file into a DataFrame
combined_data = pd.read_csv('combined_closePosition.csv')

# Sort by 'ExitTime' column
combined_data = combined_data.sort_values('ExitTime')

while True:
    # Calculating winning streaks
    winning_streaks = []
    current_winning_streak = []
    for pnl in combined_data['Pnl']:
        if pnl > 0:
            current_winning_streak.append(pnl)
        else:
            if current_winning_streak:
                winning_streaks.append(current_winning_streak)
                current_winning_streak = []
    if current_winning_streak:
        winning_streaks.append(current_winning_streak)

    # Calculating losing streaks
    losing_streaks = []
    current_losing_streak = []
    for pnl in combined_data['Pnl']:
        if pnl < 0:
            current_losing_streak.append(pnl)
        else:
            if current_losing_streak:
                losing_streaks.append(current_losing_streak)
                current_losing_streak = []
    if current_losing_streak:
        losing_streaks.append(current_losing_streak)

    # Sort the winning and losing streaks by total Pnl
    winning_streaks.sort(key=sum, reverse=True)
    losing_streaks.sort(key=sum)

    # Get the top n winning and losing streaks
    n = int(input("Enter the value of n (or -1 to exit): "))
    if n == -1:
        break

    top_winning_streaks = winning_streaks[:n]
    top_losing_streaks = losing_streaks[:n]

    # Store all the streaks in a list
    all_streaks = top_winning_streaks + top_losing_streaks

    # Append all the streaks to the existing text file or create a new file if it doesn't exist
    filename = 'combined_winning_losing.txt'
    with open(filename, 'a') as file:
        file.write(f"Top {n} winning streaks\n")
        for i, streak in enumerate(top_winning_streaks, 1):
            num_trades = len(streak)
            start_date = combined_data.loc[combined_data['Pnl'] == streak[0], 'ExitTime'].values[0]
            end_date = combined_data.loc[combined_data['Pnl'] == streak[-1], 'ExitTime'].values[0]
            pnl_streak = sum(streak)
            line = f"{i}) No. of Trades: {num_trades} Start Date: {start_date} - End Date: {end_date} Pnl of Streak: {pnl_streak}\n"
            file.write(line)

        file.write(f"Top {n} losing streaks\n")
        for i, streak in enumerate(top_losing_streaks, 1):
            num_trades = len(streak)
            start_date = combined_data.loc[combined_data['Pnl'] == streak[0], 'ExitTime'].values[0]
            end_date = combined_data.loc[combined_data['Pnl'] == streak[-1], 'ExitTime'].values[0]
            pnl_streak = sum(streak)
            line = f"{i}) No. of Trades: {num_trades} Start Date: {start_date} - End Date: {end_date} Pnl of Streak: {pnl_streak}\n"
            file.write(line)

    print(f"Top {n} winning and losing streaks saved in", filename)


Enter the value of n (or -1 to exit): 3
Top 3 winning and losing streaks saved in combined_winning_losing.txt
Enter the value of n (or -1 to exit): 4
Top 4 winning and losing streaks saved in combined_winning_losing.txt
Enter the value of n (or -1 to exit): 5
Top 5 winning and losing streaks saved in combined_winning_losing.txt
Enter the value of n (or -1 to exit): 6
Top 6 winning and losing streaks saved in combined_winning_losing.txt
Enter the value of n (or -1 to exit): 7
Top 7 winning and losing streaks saved in combined_winning_losing.txt
Enter the value of n (or -1 to exit): -1
